# This is the notebook for scraping the table from the Wikipedia page.

In [1]:
import pandas as pd # library for data analysis

## Install beautifulsoup 4 package to scrape the webpage

In [2]:
#pip install beautifulsoup4

## Install lxml package for xml parsing with BeautifulSoup

In [3]:
#pip install lxml

## Save wikipedia page into page variable and parse it using BeautifulSoup

In [4]:
#import library to query webpage of interest
from urllib.request import urlopen
#specifying page of interest
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#save the HTML of the site within the page variable
page = urlopen(wiki)
#import library to parse HTML from page
from bs4 import BeautifulSoup
#parse data from "page" and save to new variable "soup"
soup = BeautifulSoup(page, 'lxml')

## By inspecting the source of the wikipedia page, we know we want to scrape the table with the class "wikitable sortable"

In [5]:
#pinpointing the location of the table and its contents
My_table = soup.find('table',{'class':'wikitable sortable'})

In [6]:
#creating lists for each of the columns I know to be in my table.
A=[]
B=[]
C=[]

## For each row (with html code tr), append the values of each element (html code td) into the columns 

In [7]:
for row in My_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

## Define the headers of dataframe columns

In [8]:
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighbourhood']=C

## Remove rows which contain "Not assigned" in the column Borough

In [9]:
df=df[~df.Borough.str.contains('Not assigned')]

## When scrapping the table, found that the line break indicators were read as text. therefore, remove "\n" using rstrip

In [10]:
df['Neighbourhood'] = df['Neighbourhood'].map(lambda x: x.rstrip('\n'))

## For each row containing "Not assigned" in the Neighbourhood column, use the value in the Borough column

In [11]:
for i, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

## For each Borough, join the neighbourhoods in the same row and separated by ","

In [15]:
strJoin = lambda x:",".join(x.astype(str))     
new_df = df.groupby(['Postcode','Borough']).agg({"Neighbourhood":strJoin}).reset_index()

In [18]:
new_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## download geospatial data

In [19]:
geo_url = 'https://cocl.us/Geospatial_data'
df1 = pd.read_csv(geo_url)

## rename column name to match geospatial data in preparation for merging

In [20]:
new_df.rename(columns={'Postcode':'Postal Code'}, inplace=True)
new_df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Merge processed dataframe with geospatial data on "Postal Code"

In [23]:
results = new_df.merge(df1,on="Postal Code")
results.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
